Drive Mounth

In [ ]:
#colab
# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

Install and import Libraries

In [ ]:
!pip install git+https://github.com/librosa/librosa
!pip install np_utils
!pip install keras
!pip install pyeasyga

import librosa
import numpy as np
import pandas as pd
import random
import time
import glob
import os
import tensorflow as tf
import np_utils
import pyeasyga

from pyeasyga import pyeasyga
from os import listdir
from os.path import isfile, join

from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import MeanShift
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

Used dataset https://www.kaggle.com/c/freesound-audio-tagging/data

**Load Data**

In [ ]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        fp="drive/app/TUDiplomna/audio/"+fp
        X,sr = librosa.load(fp)
        raw_sounds.append(X)
    return raw_sounds

start = time.time()
mypath="drive/app/TUDiplomna/audio/"
labelsN=["Acoustic_guitar", "Applause", "Bark", "Bass_drum", "Burping_or_eructation", "Bus", "Cello", "Chime", "Clarinet", "Computer_keyboard"]
onlyfiles = []
tr_labels=pd.read_csv("drive/app/TUDiplomna/train.csv" , error_bad_lines=False)
for f in range(len(tr_labels)):
  if tr_labels.iloc[f].label in labelsN:
    onlyfiles.append(tr_labels.iloc[f].fname)
end = time.time()
print("Time for data loading:")
print(end - start)
print("seconds")
random.shuffle(onlyfiles,random.random) #mix files
train_files= onlyfiles[:500] 
test_files= onlyfiles[500:600]

#Feature Extraction

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast

def parse_audio_files(files_names):
    features, labels = np.empty((0,187)), np.empty(0)
    counter=0
    for fn in files_names:
      try:
        mfccs, chroma, mel, contrast = extract_feature("drive/app/TUDiplomna/audio/"+fn)
      except Exception as e:
        counter=counter+1
        continue
      ext_features = np.hstack([mfccs,chroma,mel,contrast])
      features = np.vstack([features,ext_features])
      labels = np.append(labels, fn)
    #print(counter)
    return features,labels

In [ ]:
start = time.time()
tr_features, tr_files= parse_audio_files(train_files)
ts_features, ts_files = parse_audio_files(test_files)
end = time.time()
print("Time for extracting test and train features:")
print(end - start)
print("seconds")

In [ ]:
def lablesExtraction(files,labels):
  labelsSet=[]
  for i in range(0, len(files)):
    for j in range(0, 9472):#check where this num come from
      if files[i]==labels['fname'].values[j]:
        label=labels['label'].values[j]
        if label != "Acoustic_guitar" and label != "Applause" and label != "Bark" and label != "Bass_drum" and label != "Bus":
          labelsSet.append('None')
        else:m
          labelsSet.append(label)
        break
  return labelsSet

def oneHotEncodder(labels):
  encoder = LabelEncoder()
  encoder.fit(labels)
  encoded_Y = encoder.transform(labels)


In [ ]:
start = time.time()

tr_features, tr_files= parse_audio_files(train_files)
ts_features, ts_files = parse_audio_files(test_files)

X_train=pd.DataFrame(tr_features)
labels_train=lablesExtraction(tr_files,tr_labels)
Y_train=oneHotEncodder(labels_train)
#pd.DataFrame(tr_labels)

X_test=pd.DataFrame(ts_features)
labels_test=lablesExtraction(ts_files,tr_labels)
Y_test=oneHotEncodder(labels_test)

end = time.time()
print("Time for extracting test and train features and labels:")
print(end - start)
print("seconds")

#**Classification**


In [ ]:
def baseline_model():
	# create model -Baseline: 53.72% (15.21%)
	model = Sequential()
	model.add(Dense(187, input_dim=187, activation='relu'))
	model.add(Dense(6, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [ ]:
def nn_dropout():#Baseline: 75.87% (9.01%)
  model = Sequential()
  model.add(Dense(18, input_dim=187, init='uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(18, init='uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(6, init='uniform', activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
start = time.time()
model = baseline_model()
# Fit the model
model.fit(X_train, Y_train, epochs=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  one layered perceptron")
print(end - start)
print("seconds")

In [ ]:

start = time.time()
model = Sequential()
model.add(Dense(18, input_dim=187, activation='relu'))
model.add(Dense(18, activation='relu'))
model.add(Dense(6, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history=model.fit(X_train, Y_train, epochs=150, batch_size=10, verbose=0)#history
end= time.time()

print("compile time in sec:")
print(end-start)
# evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  two layers perceptron")
print(end - start)
print("seconds")

In [ ]:
start = time.time()
model = nn_dropout()
# Fit the model
model.fit(X_train, Y_train, epochs=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
end = time.time()
print("Time for executing  two layers perceptron with drop out")
print(end - start)
print(" in seconds")

In [ ]:
start = time.time()
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, labels_train)
# Use score method to get accuracy of model
score = logisticRegr.score(X_test,labels_test)
print("Accuracy:")
print(score)
end = time.time()
print("Time for executing linear regression model")
print(end - start)
print(" in seconds")

In [ ]:
start = time.time()
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(multi_class=multinomial,solver=newton-cg)
logisticRegr.fit(X_train, labels_train)
# Use score method to get accuracy of model
score = logisticRegr.score(X_test,labels_test)
print("Accuracy:")
print(score)
end = time.time()
print("Time for executing linear regression model")
print(end - start)
print(" in seconds")

In [ ]:
st= time.time()
classifier = MultinomialNB()
classifier.fit(X_trai, Y_train)
core=classifier.score(X_test, Y_test)
print("Mean accuracy on the given test data and labels.")
print(score)
print('Metrics for Multinominal Naive bayes classificator ')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
st= time.time()
classifier = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=1)
classifier.fit(X_trai, Y_train)
core=classifier.score(X_test, Y_test)
print("Mean accuracy on the given test data and labels.")
print(score)
print('Metrics for Random Forest classificator')
print ('Total score: '+str(np.mean(scores)))
print ('Total time: '+str(int(time.time()-st)))

In [ ]:
#RNN
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Dense(64, input_dim=187, init='uniform', activation='relu', dropout = 0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

#Clusterization

In [ ]:
print(X)


In [ ]:
start = time.time()
#https://scikit-learn.org/stable/modules/clustering.html#mean-shift
clustering = MeanShift()
cluster_labels = clustering.fit_predict(X)
silhouette_avg = silhouette_score(X, cluster_labels)
print("The average silhouette_score is :", silhouette_avg)
end = time.time()
print("Time for executing mean shift model")
print(end - start)
print(" in seconds")

In [ ]:

def fitness(individual, data):
    temp=0.77;
    cNum=1
    for x in data:
      if temp / float(2.45) > x['score']:
        temp=x['score']
        cNum=x['clusterNum']
    return cNum


In [ ]:
start = time.time()
#KMeans(random init)
data=[];
for i in range(2,20):
  cluster_labels = KMeans(n_clusters=i, init='random', random_state=10).fit_predict(X)
  data.append({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
  
# initialise the GA with data
ga = pyeasyga.GeneticAlgorithm(data)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print ga.best_individual()
end = time.time()
print("Time for executing Kmeans with GA")
print(end - start)
print(" in seconds")

In [ ]:
start = time.time()
#KMeans++
data=[];
for i in range(2,20):
  cluster_labels = KMeans(n_clusters=i, init='random').fit_predict(X)
  data.append({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
  
# initialise the GA with data
ga = pyeasyga.GeneticAlgorithm(data)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print ga.best_individual()
end = time.time()
print("Time for executing Kmeans++ with GA")
print(end - start)
print(" in seconds")